# Twitter Sentiment Analysis

### Dataset source : Sentiment140

The data is a CSV with emoticons removed. Data file format has 6 fields:  
0 - the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)  
1 - the id of the tweet (2087)  
2 - the date of the tweet (Sat May 16 23:58:44 UTC 2009)  
3 - the query (lyx). If there is no query, then this value is NO_QUERY.  
4 - the user that tweeted (robotickilldozr)  
5 - the text of the tweet (Lyx is cool) 

## Disclaimer :  
### In the training set, we only use Positive or Negative scored tweets. So we make the asumption that all tweets are either positive or negative, and not neutral.

#### Imports

In [1]:
# General imports

import pandas as pd
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

# Words analysis oriented imports

# Natural Language ToolKit
import nltk  
# If you uncomment, it will launch an installation popup. Go to the "Models" tab and select "punkt" from the "Identifier" column. Then click "Download" and it will install the necessary files.
#nltk.download()  
from nltk.stem import PorterStemmer # For words normalization
from sklearn.feature_extraction.text import CountVectorizer # For words frequency
from sklearn.feature_extraction.text import TfidfTransformer # For words weighting

#### Read CSV

In [2]:
df = pd.read_csv("trainingandtestdata/training.1600000.processed.noemoticon.csv", header=None, encoding = "ISO-8859-1")

df.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


#### Preprocessing

a/ Conversion into lowercase

In [3]:
cols = ['sentiment_score', 'id', 'date', 'query', 'author', 'tweet']

# Rename the columns
df.columns = cols

# Convert all tweets in lowercase
df['tweet'] = df.tweet.map(lambda x: x.lower())  

b/ Feature reduction - @username, urls, letter occurences, and punctuation handling

In [4]:
# Replace all the @username by the token 'USERNAME'
df.replace('\@\w+', 'USERNAME', regex=True, inplace=True)

# Replace all the urls by the token 'URL'
df.replace(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', 'URL', regex=True, inplace=True)

# Replace any letter occurring more than two times in a row with two occurrences. Ex : huuuungry becomes huungry.
# It's important to keep two occurences as they need to stay separated from the words with one occurence, because the sentiment behind a word with multiple character occurences may be stronger (positive or negative)
df.replace(r'(.)\1{2,}', r'\1\1', regex=True, inplace=True)

# Remove punctuation
df['tweet'] = df.tweet.str.replace('[^\w\s]', '') 

df.head()

,sentiment_score,id,date,query,author,tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,USERNAME URL aww thats a bummer you shoulda ...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he cant update his facebook by t...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,USERNAME i dived many times for the ball manag...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,USERNAME no its not behaving at all im mad why...


c/ Normalization of the tweets

In [5]:
# Tokenizer
df['tweet'] = df['tweet'].apply(nltk.word_tokenize) 

# Word stemming
# Normalize our text for all variations of words carry the same meaning, regardless of the tense
stemmer = PorterStemmer()
df['tweet'] = df['tweet'].apply(lambda x: [stemmer.stem(y) for y in x]) 

df.head()

,sentiment_score,id,date,query,author,tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"[usernam, url, aww, that, a, bummer, you, shou..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,"[is, upset, that, he, cant, updat, hi, faceboo..."
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,"[usernam, i, dive, mani, time, for, the, ball,..."
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,"[my, whole, bodi, feel, itchi, and, like, it, ..."
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"[usernam, no, it, not, behav, at, all, im, mad..."


d/ Words frequency calculation in the tweets

In [6]:
# Convert the list of words into space-separated strings
df['tweet'] = df['tweet'].apply(lambda x: ' '.join(x))

# Transform the data into occurences
count_vect = CountVectorizer()  
counts = count_vect.fit_transform(df['tweet'])

# Use of Term Frequency Inverse Document Frequency, more known as tf-idf
# The tf–idf value increases proportionally to the number of times a word appears
transformer = TfidfTransformer().fit(counts)
counts = transformer.transform(counts)  

#### Train the model

Training

In [7]:
X_train, X_test, y_train, y_test = train_test_split(counts, df['sentiment_score'], test_size=0.2, random_state=42)  

# For text classification problems, the Multinomial Naive Bayes Classifier is well-suited
model = MultinomialNB().fit(X_train, y_train) 

Prediction

In [8]:
predicted = model.predict(X_test)

#### Calculate the accuracy of the model

In [9]:
accuracy_score(y_test, predicted, normalize=True)
#print(np.mean(predicted == y_test)) 

0.77034375

In [10]:
# Print the confusion matrix
print(confusion_matrix(y_test, predicted)) 

[[125967  33527]
 [ 39963 120543]]


#### Author : Thibaut BREMAND  
- thibaut.bremand [at] gmail.com
- https://github.com/ThibautBremand

#### Sources :  
- https://cs.stanford.edu/people/alecmgo/papers/TwitterDistantSupervision09.pdf : The original research paper which inspered me
- http://help.sentiment140.com  : The original training dataset  
- https://stackabuse.com/the-naive-bayes-algorithm-in-python-with-scikit-learn/ : Explanation of the methodology. 
- https://www.datacamp.com/community/tutorials/naive-bayes-scikit-learn : More details about the Naive Bayes classification